# Training covid models
### This notebook is an example usage of how to use the model alongside the covid-data-collector in order to train, evaluate and test the model
#### In this notebook you will find example usages on how to use the core functionalities of the model 

#### Import third party modules, and also the data_collector: covid19_genome and the model module

In [14]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Uncomment to disable GPU
import glob

from model import Model, DatasetName, load_model, remove_model

__ORIG_WD__ = os.getcwd()

os.chdir(f"{__ORIG_WD__}/../data_collectors/")
from covid19_genome import Covid19Genome

os.chdir(__ORIG_WD__)


#### Create a model, or try to load it, if it was already have been created.

In order to use the model, the first thing you have to do is provide it with a dataset (with the help of the data_collector). In the following cell you are provided with an example that create the dataset.

You should note that when you are creating the dataset, you are passing the dataset type. You can obtain the available dataset types in the system by calling the model class function ```get_ds_types()```

In [15]:
model_name = "covid19-1024examples"

try:
    model = load_model(model_name)
except Exception:
    covid19_genome = Covid19Genome()
    lineages = covid19_genome.getLocalLineages(1024)
    lineages.sort()
    dataset = []
    def get_dataset():
        for lineage in lineages:
            dataset.append((lineage, covid19_genome.getLocalAccessionsPath(lineage)))
        return dataset

    portions = {
        DatasetName.trainset.name: 0.8,
        DatasetName.validset.name: 0.1,
        DatasetName.testset.name: 0.1
    }

    dataset = get_dataset()
    model = Model(model_name)
    model.create_datasets(model.get_ds_types()[0], dataset, portions)

After you have created the model, and created its datasets. You can check which neural network structures is available. You can do that by calling the model class function ```get_ml_model_structure()```.

After you see all the ml_model structures available in the system, you can check which hyper parameters are needed to define each and every ml_model structure. This is done by calling the model class function ```get_ml_model_structure_hps()```. The ```get_ml_model_structure_hps()``` will return which hps are required, and what it their type.

In [16]:
print(model.get_ml_model_structures())
print(model.get_ml_model_structure_hps(model.get_ml_model_structures()[0]))

['VitStructure', 'IreneStructure', 'SandyStructure']
{'d_model': 'required', 'd_val': 'required', 'd_key': 'required', 'd_ff': 'required', 'heads': 'required', 'dropout_rate': 'optional', 'regularizer': 'optional', 'initializer': 'optional', 'activation': 'optional', 'encoder_repeats': 'required', 'labels': 'required'}


You can also see which properties help define the current type of dataset by calling to the model class function ```get_ds_props()``` This function could be called only after the dataset have been succesfully created. This function will return the properties of the dataset as well as their values.

In [17]:
print(model.get_ds_props())

{'coverage': 2, 'substitution_rate': 0.005, 'insertion_rate': 0.025, 'deletion_rate': 0.025, 'read_length': 128, 'frag_len': 128, 'num_frags': 256}


A use case of the system with the VitStructure model and the minhash genome datasets (a.k.a. mh_genome_ds).

In the mh_genome_ds the coverage is a dataset property that sets the genome coverage rate.

In the VitStructure, the model_depth is the number of transformer encoders.

In this example use-case these two parameters will help us define a neural network that will be trained on the dataset (with the current coverage rate)

In [18]:
coverage = 2
ml_model_depth = 6
sequencer_instrument = "pacbio"

In [19]:
sequencer_instrument_to_error_profile_map = {
    "illumina": {
        "substitution_rate": 0.005,
        "insertion_rate": 0.001,
        "deletion_rate": 0.001
    },
    "ont": {
        "substitution_rate": 0.01,
        "insertion_rate": 0.04,
        "deletion_rate": 0.04
    },
    "pacbio": {
        "substitution_rate": 0.005,
        "insertion_rate": 0.025,
        "deletion_rate": 0.025
    },
    "roche": {
        "substitution_rate": 0.002,
        "insertion_rate": 0.01,
        "deletion_rate": 0.01
    }
}

def get_model_name(ml_model_depth, coverage, sequencer_instrument):
    if not sequencer_instrument in sequencer_instrument_to_error_profile_map:
        raise Exception(f"Invalid sequencer instrument: {sequencer_instrument}")
    return f"vit.{ml_model_depth}.{coverage}x.{sequencer_instrument}"

ml_model_name = get_model_name(ml_model_depth, coverage, sequencer_instrument)
print(ml_model_name)

vit.6.2x.pacbio


#### Adding a new neural network

In this cell we will create an ml_model with the required hps (and also optional) as outputted earlier.

In [20]:
newly_added = True
# model.remove_ml_model(ml_model_name)
try:
    model.add_ml_model(ml_model_name, hps={
        "structure": model.get_ml_model_structures()[0],
        "d_model": model.get_ds_props()["frag_len"],
        "d_val": 128,
        "d_key": 128,
        "heads": 8,
        "d_ff": 1024+256,
        "labels":  len(model.get_labels()),
        "activation": "relu",
        "optimizer": {
            "name": "AdamW",
            "params": {
                "learning_rate": 0.001,
            },
        },
        "encoder_repeats": ml_model_depth,
        "regularizer": {
            "name": "l2",
            "params": {
                "l2": 0.0001
            }
        },
        "dropout_rate": 0.1,
    })
except:
    newly_added = False
    print("Model already exists")

Model already exists


In [21]:
models = model.list_ml_models()
print(models)

['vit.2.4x.pacbio', 'vit.6.2x.pacbio', 'vit.2.2x.illumina', 'vit.2.1x.illumina', 'meow', 'vit.1.2x.pacbio', 'vit.1.4x.roche', 'vit.2.4x.roche', 'sandy.1.2x.pacbio', 'vit.2.2x.pacbio', 'vit.2.2x.roche', 'irene.1.2x.pacbio', 'vit.2.1x.roche', 'vit.3.2x.pacbio', 'vit.2.1x.pacbio', 'vit.2.4x.illumina', 'vit.3.1x.illumina', 'vit.1.4x.illumina']


In [22]:
#if newly_added:
#    assert False, "Please consider doing transfer learning"
# model.transfer(get_model_name(ml_model_depth-3, coverage, sequencer_instrument), ml_model_name)

In [23]:
model.change_ml_hps(ml_model_name, {
#     "regularizer": {        
#         "name": "l2",
#         "params": {
#             "l2": model.get_ml_hps(ml_model_name)["regularizer"]["params"]["l2"] * 0.5,
#         },
#    },
#    "optimizer": {
#        "name": "AdamW",
#        "params": {
#             "learning_rate": model.get_ml_hps(ml_model_name)["optimizer"]["params"]["learning_rate"] * 0.5,
#         },
#     },
    # "dropout_rate": model.get_ml_hps(ml_model_name)["dropout_rate"] * 0.5,
})

#### Updating the dataset coverage

In [24]:
model.update_ds_props({
    "coverage": coverage,
    } | sequencer_instrument_to_error_profile_map[sequencer_instrument])

In [25]:
model.get_model_summary(ml_model_name)

Model: "vit_structure_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            multiple                  5         
                                                                 
 encoder_18 (Encoder)        multiple                  857088    
                                                                 
 encoder_19 (Encoder)        multiple                  857088    
                                                                 
 encoder_20 (Encoder)        multiple                  857088    
                                                                 
 encoder_21 (Encoder)        multiple                  857088    
                                                                 
 encoder_22 (Encoder)        multiple                  857088    
                                                                 
 encoder_23 (Encoder)        multiple              

#### Setting dataset batch size and training

In [26]:
model.set_ds_batch_size(120)
model.train(ml_model_name, epochs=1000)

Epoch 1/1000


2023-09-18 09:15:10.074867: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:273] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-09-18 09:15:10.077048: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at xla_ops.cc:503 : INTERNAL: libdevice not found at ./libdevice.10.bc
2023-09-18 09:15:10.127052: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:273] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-09-18 09:15:10.131793: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at xla_ops.cc:503 : INTERNAL: libdevice not found at ./libdevice.10.bc
2023-09-18 09:15:10.176856: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:273] libdevice is required by this HLO module but was not found at ./libdevice.10.bc
2023-09-18 09:15:10.179979: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at xla_ops.cc:5

InternalError: Graph execution error:

Detected at node 'AdamW/StatefulPartitionedCall_99' defined at (most recent call last):
    File "/home/zuherj/miniconda3/envs/vital/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/zuherj/miniconda3/envs/vital/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/zuherj/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/zuherj/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/home/zuherj/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 724, in start
      self.io_loop.start()
    File "/home/zuherj/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/zuherj/miniconda3/envs/vital/lib/p

In [ ]:
import tensorflow as tf
print(tf.__version__)